Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "NOÉ FUENTES PÉREZ"
COLLABORATORS = ""

---

# El ordenador juega al Wordle

En este segundo apartado vamos a hacer un programa que juege al wordle. El ordenador va a tratar de encontrar la palabra que nosotros hayamos escogido. Para no hacer trampas debemos escoger una palabra que se encuentre el el fichero <code>lemario.txt</code>. Vamos a coger una palabra de 5 letras, si quisieramos cambiar esa longitud deberíamos cambiar las constantes de forma adecuada. Por ahora vamos a coger las constantes que ya teníamos definidas.

In [ ]:
LETTER_IN_POS = 0
LETTER_IN_NO_POS = 1
LETTER_NOT_IN = 2
MAX_ATTEMPTS = 6
WORD_LENTGH = 5

De entrada, tendremos que usar las funciones que ya teníamos en el apartado anterior.

In [ ]:
LETTER_IN_POS = 0
LETTER_IN_NO_POS = 1
LETTER_NOT_IN = 2
MAX_ATTEMPTS = 6
WORD_LENTGH = 5

def read_words(filename, wlen):
    with open(filename, encoding="utf8") as f:
        lst = []
        r = f.readline()
        while r != '':
            if len(r.strip('\n')) == wlen:
                lst.append(r.strip('\n'))
            r = f.readline()
        return lst


def get_clues(secret, user_word):
    """ 
    This funcction returns the clues for the word user_word with respect secret. 
    For each letter it returns 
      - LETTER_IN_POS if the letter is in secret in the same position. 
      - LETTER_IN_NO_POS if the letter is in secret but in another position.
      - LETTER_NOT_IN if the letter is not in secret.
    The function returns a list with the corresponding values.
    Example
    secret = 'GOTAS'
    user_word = 'GALGO'
    returns -> [LETTER_IN_POS, LETTER_IN_POS, LETTER_NOT_IN, LETTER_IN_NO_POS, LETTER_IN_NO_POS]
    """
    secret = secret.lower()
    user_word = user_word.lower()
    replacements = (("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"))
    for a, b in replacements:
        secret = secret.replace(a, b)
        user_word = user_word.replace(a, b)
    if len(user_word) != len(secret):
        raise Exception(f'Debes introducir una palabra de longitud {len(secret)}')
    clues = []
    s = [m for m in secret]
    uw = [n for n in user_word]
    for i in range(len(s)):
        if s[i] == uw[i]:
            clues.append(0)
        elif uw[i] in s:
            clues.append(1)
        else:
            clues.append(2)
    return clues

def are_all_in_pos(answer):
    """
    This function returns a boolean.
    It returns True if all elements in the list answer are LETTER_IN_POS.
    """
    return answer == [0] * len(answer)



## El algoritmo

Veamos someramente el algoritmo que va a seguir nuestro programa. Tenemos todas las posibles palabras almacenadas en una lista. El programa escoge la primera palabra de la lista y la propone como posible candidata. Pregunta por la respuesta y la almacena. A continuación, siguiendo por la siguiente palabra de la lista, busca la primera palabra que es <em>compatible</em> con la respuesta. La propone y la almacena en la lista la palabra y la respuesta. En todo momento lo que se hace es buscar a partir de la última palabra propuesta, la siguiente que es <em>compatible</em> con las respuestas almacenadas. Pero ¿qué significa exactamente ser compatible? Una palabra es compatible con todas las respuestas, si asumiendo que ella es la palabra secreta, da la misma respuesta a las palabras almacenadas.
El programa llevará una lista con las palabras que han sido propuestas y sus respuestas. Por ejemplo, si tenemos las siguientes palabras con sus respuestas


Palabra | Respuesta
--------|------------------
'ababa' |  [0, 2, 1, 2, 0]
'acema' |  [0, 2, 0, 2, 0]

La palabra 'acuma' no es compatible por que la resupuesta que daría a 'acema' es <code>[0,0,2,0,0]</code> en lucar de <code>[0,2,0,2,0]</code>. La palabra 'aleda' sí que es compatible. 



# El programa principal

Para desarrollar el programa empezaremos otra vez por la función principal del programa. Será parecida a la del programa anterior. Necesitamos el almacén de palabras y respuestas para poder averigüar la siguiente palabra compatible, a esa variable la llamaremos <code>storage</code>. Luego necesitamos la lista de palabras y la posición actual para poder seguir las palabras en orden.                           

En cada iteración buscamos la siguiente palabra compatible y almacenamos la nueva respuesta en el almacén. Por último un pequeño detalle, en un juego <em>perfecto</em>, el fácil demostar que el algoritmo anterior llega a la solución. El problema es que el da las respuestas y bien se puede equivocar o directamente intentar hacer trampas. En esos casos, es posible que no exista la siguiente palabra compatible. 


In [ ]:
def computer(words_file):
    pos = 0
    words = read_words(words_file, WORD_LENTGH)
    storage = []
    game_cont = True
    while game_cont:
        pos = next_candidate(storage, words, pos)
        if pos<len(words):
            candidate = words[pos]
            answer = get_answer(candidate)
            add_answer(storage, candidate, answer)
            pos += 1
        game_cont = pos<len(words) and not are_all_in_pos(answer)
    if pos==len(words):
        print('You have chosen an illegal word or have given an incorrect answer')
    else:
        print(f'The secret word is {candidate}')
    

El programa anterior no funciona porque falta la definición de las varias funciones: <code>next_candidate</code>, <code>add_answer</code> y <code>get_answer</code>. Las funciones <code>read_words</code> y <code>are_all_in_pos</code> ya están definidas de la versión anterior. 

En primer lugar daremos la función <code>add_answer</code>. Como parámetros de entrada hay una palabra candidata y debe devolver una lista de 0, 1 y 2 de la misma longitud de la palabra con la respuesta del usuario. El usuario responde con una secuencia de números separada por espacios. 

Se debe comprobar que la respuesta que da el usuario es la correcta, si no lo es se debe preguntar de nuevo al usuario. 

No te olvides de escribir la documentación de la función.

In [ ]:
def get_answer(candidate):
    candidate = candidate.lower()
    print(candidate)
    a = input("Your answer: ")
    a = a.lower()
    k=0
    while k<5 and (len(a) != len(candidate) or a == ''):
        print('You have entered an incorrect answer. Please try again')
        a = input("Your answer: ")
        a = a.lower()
        k+=1
    if k==5:
        print('Please, stop cheating. You are a real pain in the ass to deal with.')
        return
    b = get_clues(a, candidate)
    return b

En la casilla de que sigue comprueba que tu programa funciona correctamente. Escribe tus propias pruebas

In [ ]:
### BEGIN HIDDEN TEST
input_ori = input
class MyInput():
    def __init__(self, answers):
        self.pos = 0
        self.answers = answers
    def input(self, msg):
        if self.pos<len(self.answers):
            ans = self.answers[self.pos]
            self.pos += 1
        else:
            ans = ''
        return ans
try:
    test = ['0 1 0 1 2']
    print(f'checking {test}....', end='')
    myinput = MyInput(test)
    input = myinput.input
    answer = get_answer("abcde")
    assert answer==[0, 1, 0, 1, 2], f'La respuesta es {answer}'
    print('ok')

    test = ['0 0 1 2', '0 1 0 1 2']
    print(f'checking {test}....', end='')
    myinput = MyInput(test)
    input = myinput.input
    answer = get_answer("abcde")
    assert answer==[0, 1, 0, 1, 2], f'La respuesta es {answer}'
    print('ok')


    test = ['0 0 0 3 2', '0 1 0 1 2']
    print(f'checking {test}....', end='')
    myinput = MyInput(test)
    input = myinput.input    
    answer = get_answer("abcde")
    assert answer==[0, 1, 0, 1, 2], f'La respuesta es {answer}'
    print('ok')


    test = ['0 0 0 1 0 2', '0 1 0 1 2']
    print(f'checking {test}....', end='')
    myinput = MyInput(test)
    input = myinput.input    
    answer = get_answer("abcde")
    assert answer==[0, 1, 0, 1, 2], f'La respuesta es {answer}'
    print('ok')


finally:
    input = input_ori
### END HIDDEN TEST

A continuación vamos a por la función <code>add_answer</code>. Debe modifificar la variable <code>storage</code> que tiene como parámetro para que incorpore una nueva tupla con la palabra y la respuesta. El primer elemento de la tupla debe ser la palabra y la segunda la respuesta de forma que la variable storage tenga un contenido similar al siguiente

    <code>[('ababa', [0, 2, 1, 2, 0]), ('acema', [0, 2, 0, 2, 0]), ('aleda', [0, 2, 0, 2, 0]) ]</code>

Recuerda de escribir la documentación de la función

In [ ]:
def add_answer(storage, candidate, answer):
    candidate = candidate.lower()
    storage.append(tuple((candidate,answer)))
    return storage


En la casilla a continuación escribe las pruebas que hagas para ver que esa función está bien

Para realizar la función <code>next_candidate</code> debebmos partir de la posición actual y encontrar la siguiente palabra compatible con el almacen en la lista de palabras. Por ello vamos a hacer primero una función que nos diga si una palabra es compatible con las almacenadas en el almacén. 

In [ ]:
def is_compatible(storage, word):
    k = 0
    for i in range(len(storage)):
        if get_clues(word, storage[i][0]) != storage[i][1]:
            k += 1
    return k == 0

En la siguiente casilla puedes probar tu solución 

In [ ]:
print(is_compatible([('aviones', [1, 2, 0, 1, 0, 2, 2]), ('arbitro', [1, 2, 0, 2, 1, 2, 1]), ('cochazo', [1, 1, 1, 2, 1, 1, 0])],'zancudo'))
print(is_compatible([('aviones', [1, 2, 0, 1, 0, 2, 2]), ('arbitro', [1, 2, 0, 2, 1, 2, 1]), ('cochazo', [2, 0, 1, 2, 1, 1, 0])],'zancudo'))
print(is_compatible([('aviones', [1, 2, 2, 1, 1, 2, 2]), ('arbitro', [1, 2, 2, 2, 2, 2, 0]), ('cochazo', [1, 1, 1, 2, 1, 1, 0])],'zancudo'))

Ahora puedes dar la implementación de la función <code>next_candidate</code>. Fíjate en el lugar en el que se usa, te indicará los parámetros que tiene y lo que tiene que devolber. No te olvides de poner los comentarios adecuados en la definición

In [ ]:
def next_candidate(storage, words, pos):
    w = pos
    if storage:
        while w < len(words) and not is_compatible(storage, words[w]):
            w += 1
        return w
    else:
        return pos + 1

En la siguiente casilla debes probar tu solución.

In [ ]:
print(next_candidate([('banco', [2, 0, 2, 2, 0]), ('basto', [2, 0, 0, 2, 0]), ('campo', [2, 0, 2, 2, 0]), ('canto', [2, 0, 2, 2, 0]), \
                      ('cauto', [2, 0, 2, 2, 0]), ('dardo', [2, 0, 1, 2, 0]), ('falso', [2, 0, 2, 1, 0]), ('galgo', [1, 0, 2, 0, 0]), \
                      ('gasto', [1, 0, 0, 2, 0]), ('harto', [2, 0, 1, 2, 0]), ('jaleo', [2, 0, 2, 2, 0]), ('labio', [2, 0, 2, 2, 0]), \
                      ('macho', [2, 0, 2, 2, 0]), ('manto', [2, 0, 2, 2, 0]), ('pacto', [2, 0, 2, 2, 0]), ('parto', [2, 0, 1, 2, 0]), \
                      ('pasto', [2, 0, 0, 2, 0]), ('rasgo', [0, 0, 0, 0, 0]), ('sabio', [1, 0, 2, 2, 0]), ('salto', [1, 0, 2, 2, 0]), \
                      ('santo', [1, 0, 2, 2, 0]), ('tacto', [2, 0, 2, 2, 0]), ('vasto', [2, 0, 0, 2, 0]), ('yazco', [2, 0, 2, 2, 0]), \
                      ('zanco', [2, 0, 2, 2, 0])],['banco', 'basto', 'campo', 'canto', 'cauto', 'dardo', 'falso', 'galgo', \
                                                              'gasto', 'harto', 'jaleo', 'labio', 'macho', 'manto', 'pacto', 'parto', \
                                                              'pasto', 'rasgo', 'sabio', 'salto', 'santo', 'tacto', 'vasto', 'yazco', \
                                                              'zanco'],0))
def prueba():
    k=0
    l=[]
    t=[('cacto', [1, 0, 0, 0, 0]), ('canto', [1, 0, 2, 0, 0]), ('capto', [1, 0, 2, 0, 0]), ('casto', [1, 0, 2, 0, 0]), ('cauto', [1, 0, 2, 0, 0])]
    r=['balto', 'basto', 'cacto', 'canto', 'capto', 'casto', 'cauto', 'danto', 'facto', 'falto', 'fasto', 'gasto', 'harto', 'jacto', 'jauto', 'lacto', 'lasto', 'lauto', 'manto', 'masto', 'nanto', 'pacto', 'palto', 'parto', 'pasto', 'pauto', 'rapto', 'saeto', 'salto', 'santo', 'tacto', 'tanto', 'tasto', 'vasto', 'yanto']
    while k<35 and k is not False:
        s=next_candidate(t,r,k)
        if s:
            l.append(s)
        k=s
    for i in range(len(l)):
        print(f'{r[l[i]]} in position {l[i]}')
prueba()

## Programa completo

Ha llegado el meomento de hacer el programa completo. Coge la función <code>computer</code> y todas las demás y ponlas a conticunación. La ejecuciön del programa genera un fichero <code>computer.py</code>

In [ ]:
%%file computer.py

def computer(words_file):
    pos = 0
    words = read_words(words_file, WORD_LENTGH)
    storage = []
    game_cont = True
    while game_cont:
        pos = next_candidate(storage, words, pos)
        if pos < len(words):
            candidate = words[pos]
            answer = get_answer(candidate)
            add_answer(storage, candidate, answer)
            pos += 1
        game_cont = pos < len(words) and not are_all_in_pos(answer)
    if pos == len(words):
        print('You have chosen an illegal word or have given an incorrect answer')
    else:
        print(f'The secret word is {candidate}')

Ejecuta tu programa en la siguiente casilla para ver si funciona bien

In [ ]:
### BEGIN HIDDEN TEST
import sys 
import io

stdout = sys.stdout
input_ori = input
class MyInput():
    def __init__(self, answers):
        self.pos = 0
        self.answers = answers
    def input(self, msg):
        if self.pos<len(self.answers):
            ans = self.answers[self.pos]
            self.pos += 1
        else:
            ans = ''
        return ans
try:
    myinput = MyInput(['0 2 1 2 1', 
                       '0 2 2 2 1', 
                       '0 2 2 0 2', 
                       '0 2 2 0 2',
                       '0 2 2 0 0', 
                       '0 1 0 0 0', 
                       '0 0 0 0 0'])
    
    output = io.StringIO("")
    input = myinput.input
    sys.stdout = output
    computer("lemario.txt")
    assert output.getvalue() == 'The secret word is avion\n', f'"{output.getvalue()}"'

    myinput = MyInput(['0 2 1 2 1', 
                       '0 2 2 2 1', 
                       '0 2 2 0 2', 
                       '0 2 2 0 2',
                       '0 2 2 0 0', 
                       '0 1 0 0 0', 
                       '0 0 0 0 1'])
    
    output = io.StringIO("")
    input = myinput.input
    sys.stdout = output
    computer("lemario.txt")
    assert output.getvalue() == 'You have chosen an illegal word or have given an incorect answer\n', f'"{output.getvalue()}"'


    
finally:
    input = input_ori
    sys.stdout = stdout
### END HIDDEN TEST

## Ordenador contra sí mismo

Por último haz un programa que juegue contra sí mismo. Haz una función llamada
<code>computer_vs_computer</code> de forma que el ordenador juegue contra sí mismo, sin hacer trampas. La función como las anteriores debe contener un el nombre de un fichero de palabras. Debe devolver una tupla, el primer elemento de la tupla debe ser la palabra seleccionada, el segundo una lista con las todas las respuestas calculadas. Para seleccionar la palabra usa la función <a href='https://docs.python.org/3.8/library/random.html#random.randint'><code>random.randint</code></a>

In [ ]:
def computer_vs_computer(filename):
    random.seed()
    with open(filename, encoding="utf8") as f:
        r = f.readlines()
        for i in range(len(r)):
            r[i] = r[i].strip('\n')
        words = r
    secret = words[random.randint(0, len(words) - 1)]
    lords = read_words(filename, len(secret))
    firstword = lords[random.randint(0, len(lords) - 1)]
    storage = [(firstword, get_clues(secret, firstword))]
    game_cont = True
    pos = 0
    attempts = 1
    while game_cont:
        pos = next_candidate(storage, lords, pos)
        if pos < len(lords):
            candidate = lords[pos]
            answer = get_clues(secret, candidate)
            storage = add_answer(storage, candidate, answer)
        attempts += 1
        game_cont = pos < len(lords) and not are_all_in_pos(answer) and attempts < MAX_ATTEMPTS
    if pos == len(lords):
        print('The computer just cheated! Or, maybe, the code\'s wrong')
    else:
        return tuple((secret, storage))

Ejecuta en la casilla siguientes pruebas para ver que realmente funciona.

In [ ]:
computer_vs_computer('lemario.txt')

Por último, añade la función <code>computer_cvs_computer</code> al programa <code>computer.py</code>.